
This notebook reproduces the analyses presented in the study:  
**"Dissecting Healthcare Accessibility Shifts during COVID-19 using Mobility Data"**

We integrate large-scale mobility data with interpretable machine learning to examine year-over-year changes in healthcare accessibility between 2019 and 2022, focusing on four service types:

- Adult Primary Care  
- Pediatric Primary Care  
- Emergency Room Visits  
- Urgent Care Visits  

We use:
- Utilization-based accessibility models
- CatBoost regressors
- SHAP values for model interpretation


## 1. Accessibility for healthcare four category from 2019-2023

Use access_cal.py to generate the accessibility data for each CBG for four healthcare category for year 2019-2023 seperatly.

In [4]:
import pandas as pd

adult_primary_care_19 = pd.read_csv('/home/disk/hao/poi_v2_healthcare/weekly/accessibility/acess_cal_v2/2019_Adult Primary Care_access.csv')

print(adult_primary_care_19['accessibility'].head)

<bound method NDFrame.head of 0       0.195882
1       0.112438
2       0.170332
3       0.386861
4       0.220392
          ...   
5507    0.360792
5508    0.300502
5509    0.264490
5510    0.132621
5511    0.163055
Name: accessibility, Length: 5512, dtype: float64>


## 2. Year-over-Year Accessibility Visualizations for four healthcare (2019–2022)

In [5]:
import numpy as np # Import numpy to handle inf values
import os
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
def visualize_accessibility_difference(yearA, yearB, category):
    """
    Calculates and visualizes the difference in accessibility for a given category
    between two specified years.

    Args:
        yearA (int): The first year.
        yearB (int): The second year.
        category (str): The healthcare category (e.g., "Hospital", "Pharmacy").
    """
    print(f"Calculating difference for {category}: {yearA} - {yearB}")

    # Construct filenames
    file_template = '{year}_{category}_access.csv'
    filenameA = file_template.format(year=yearA, category=category)
    filenameB = file_template.format(year=yearB, category=category)
    # filenameA = f"{yearA}_{category}_access.csv"
    # filenameB = f"{yearB}_{category}_access.csv"

    filepathA = os.path.join(input_dir, filenameA)
    filepathB = os.path.join(input_dir, filenameB)
    # Check if files exist
    if not os.path.exists(filepathA):
        print(f"Error: File not found - {filepathA}")
        return
    if not os.path.exists(filepathB):
        print(f"Error: File not found - {filepathB}")
        return

    # Load accessibility data
    access_dataA = pd.read_csv(filepathA)
    access_dataB = pd.read_csv(filepathB)

    # Replace inf and -inf values with 0 in the 'accessibility' column
    access_dataA['accessibility'] = access_dataA['accessibility'].replace([np.inf, -np.inf], 0)
    access_dataB['accessibility'] = access_dataB['accessibility'].replace([np.inf, -np.inf], 0)

    # Ensure CBG columns are strings and padded
    access_dataA['cbg'] = access_dataA['cbg'].astype(str).str.zfill(12)
    access_dataB['cbg'] = access_dataB['cbg'].astype(str).str.zfill(12)

    # Merge data with geographic data
    # Use a common base (geo_data) to ensure all GEOIDs are present
    merged_diff = geo_data.copy()
    merged_diff = merged_diff.merge(access_dataA[['cbg', 'accessibility']], left_on='GEOID', right_on='cbg', how='left').drop(columns='cbg')
    merged_diff = merged_diff.rename(columns={'accessibility': 'accessibility_A'})
    merged_diff = merged_diff.merge(access_dataB[['cbg', 'accessibility']], left_on='GEOID', right_on='cbg', how='left').drop(columns='cbg')
    merged_diff = merged_diff.rename(columns={'accessibility': 'accessibility_B'})

    # Calculate the difference, handling potential NaNs (e.g., if a CBG is missing in one year)
    # Fill NaN accessibility scores with 0 before calculating difference
    merged_diff['accessibility_A'] = merged_diff['accessibility_A'].fillna(0)
    merged_diff['accessibility_B'] = merged_diff['accessibility_B'].fillna(0)

    merged_diff['accessibility_difference'] = merged_diff['accessibility_A'] - merged_diff['accessibility_B']

    # Determine color scale limits using the 99th percentile of absolute differences
    abs_diff = merged_diff['accessibility_difference'].abs()

    # Handle the case where all differences are zero or NaN after filling
    if abs_diff.sum() == 0:
         max_limit = 0
    else:
        # Calculate the 99th percentile of absolute differences
        max_limit = abs_diff.quantile(0.99)


    vmin = -max_limit
    vmax = max_limit

    # Check if there's any significant difference to visualize
    # This check still uses the calculated vmin/vmax which are based on the percentile
    if abs(vmax - vmin) < 1e-9: # Use a small tolerance for floating point comparison
        print(f"No significant accessibility difference found for {category}: {yearA} vs {yearB} within the 99th percentile. Skipping plot generation.")
        return

    # Create plot for difference
    fig, ax = plt.subplots(1, 1, figsize=(15, 10))
    merged_diff.plot(
        column='accessibility_difference',
        ax=ax,
        legend=True,
        legend_kwds={'label': f'Accessibility Difference ({yearA} - {yearB})', 'shrink': 0.5},
        missing_kwds={'color': 'lightgrey'},\
        cmap='coolwarm', # Use a diverging colormap
        vmin=vmin, # Set common scale min
        vmax=vmax  # Set common scale max
    )

    # Configure plot
    plt.title(f'{category} Accessibility Difference in Georgia: {yearA} vs {yearB} (99th Percentile Scale)', pad=20, fontsize=14)
    ax.axis('off')

    # Save and close
    output_filename = f'{yearA}_{yearB}_{category.replace(" ", "_")}_accessibility_difference_99percentile.png' # Appended _99percentile to filename
    output_path = os.path.join(output_dir, output_filename)
    plt.savefig(output_path, dpi=300, bbox_inches='tight')
    plt.close()

    print(f'Saved difference plot: {output_path}')

In [ ]:
input_dir = '/home/disk/hao/poi_v2_healthcare/weekly/accessibility/acess_cal_v2'
output_dir = '/home/disk/hao/poi_v2_healthcare/weekly/accessibility/acess_cal_v2/covid/difference'


geojson_path = "/home/disk/hao/poi_v2_healthcare/tl_2019_13_bg/tl_2019_13_bg.shp"
os.makedirs(output_dir, exist_ok=True)

# Load geographic data once
geo_data = gpd.read_file(geojson_path)
geo_data['GEOID'] = geo_data['GEOID'].astype(str).str.zfill(12)


In [ ]:
year_combinations = [
    (2019, 2020),
    (2020, 2021),
    (2021, 2022),
    (2022, 2023),
    (2019, 2023),
    (2022, 2020)
]

# Define the healthcare categories
categories = [
    'Adult Primary Care',
    'Emergency Room Visits',
    'Pediatric Primary Care',
    'Urgent Care Visits'
]

for yearA, yearB in year_combinations:
    for category in categories:
        visualize_accessibility_difference(yearA, yearB, category)

## (1). Adult Primary Care

We first examine spatial patterns of accessibility change in Adult Primary Care across Georgia from 2019 to 2022. Each map shows the difference in accessibility scores between two consecutive years.

- **(a)** 2020 vs. 2019  
- **(b)** 2021 vs. 2020  
- **(c)** 2022 vs. 2021


![Adult Primary Care Accessibility Change over time](pics/covid_difference_APC.png)

## (2). Emergency Room Visits

Similar visualizations for Emergency Room accessibility changes show:

- **(a)** 2020 vs. 2019  
- **(b)** 2021 vs. 2020  
- **(c)** 2022 vs. 2021


![Emergency Room Visits Accessibility Change over time](pics/covid_difference_ER.png)

## (3). Pediatric Primary Care

The maps below visualize changes in Pediatric Primary Care accessibility:

- **(a)** 2020 vs. 2019  
- **(b)** 2021 vs. 2020  
- **(c)** 2022 vs. 2021


![Pediatric Primary Care Accessibility Change over time](/pics/covid_difference_PPC.png)

## (4). Urgent Care Visits

Year-over-year accessibility shifts in Urgent Care services are illustrated below:

- **(a)** 2020 vs. 2019  
- **(b)** 2021 vs. 2020  
- **(c)** 2022 vs. 2021


![Urgent Care Accessibility Change over time](pics/covid_difference_UC.png)

## 3 Relative Accessibility Change (RAC) Patterns and Modeling Setup 
We define the target variable for our modeling as the **Relative Accessibility Change (RAC)** from 2020 to 2022 at the census block group (CBG) level for each healthcare service type.

This section walks through the following:

1. RAC definition and transformation  
2. SHAP-based feature importance  
3. Feature selection via Spearman correlation  
4. SHAP value normalization  
5. Model training and evaluation using CatBoost

RAC:

![RAC](pics/2020_2022covid_difference_all.png)

**for the modeling, run catboost0727.py file.**